### LOADING DATA

In [16]:
from langchain_community.document_loaders import PyPDFLoader


loader = PyPDFLoader(r"C:\Users\LENOVO\Desktop\ML_RAG\donnee_use_case_structuré.pdf")
data = loader.load()
print(len(data))
print(type(data))


124
<class 'list'>


In [33]:
!pip install langchain_ollama


  Obtaining dependency information for langchain_ollama from https://files.pythonhosted.org/packages/44/a1/a7dbdc39365f2f148a91724d8d52c0028cafe7dd6f0257462bc187bc4643/langchain_ollama-0.3.0-py3-none-any.whl.metadata
  Obtaining dependency information for ollama<1,>=0.4.4 from https://files.pythonhosted.org/packages/31/83/c3ffac86906c10184c88c2e916460806b072a2cfe34cdcaf3a0c0e836d39/ollama-0.4.7-py3-none-any.whl.metadata
  Using cached ollama-0.4.7-py3-none-any.whl.metadata (4.7 kB)



[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


### SPLIT THE DATA

In [30]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000,
    chunk_overlap=200
)
chunks = text_splitter.split_documents(data)

print((chunks[0].page_content))
print(type(chunks))

Données d'utilisation des clients
Univers : Lignes
Variable : LIGNE_ID
Description : Identifiant de la ligne (numéro de téléphone)
Utilisé pour : La segmentation (Oui)
Churn : Oui
Nb : 1
Univers : Lignes
Variable : LIGNE_TYP
Description : Type de ligne (prépayé ou postpayé)
Utilisé pour : La segmentation (Oui)
Churn : Non
Nb : 1
Univers : Lignes
Variable : LIGNE_CUST_ANC
Description : Ancienneté de la ligne en jours (date courante - date d'activation de la ligne)
Utilisé pour : La segmentation (Oui)
Churn : Oui
Nb : 1
Univers : Lignes
Variable : LIGNE_DATE_ACT
Description : Date d'activation de la ligne
<class 'list'>


### Vector store

In [27]:
from langchain_nomic import NomicEmbeddings
from langchain_community.vectorstores import FAISS
import os

os.environ["NOMIC_API_KEY"] = "nk-uC7cI5olWsQ8GFL5d_y54N5zpNlx9vH86SgOieRzBWA"
embeddings = NomicEmbeddings(model="nomic-embed-text-v1.5")


vector_store = FAISS.from_documents(documents=chunks, embedding=embeddings)
print(type(vector_store))

<class 'langchain_community.vectorstores.faiss.FAISS'>


### Retriever 

In [28]:
retriever = vector_store.as_retriever()


### QUERY

In [ ]:
# -----------QUERY--------------
query = "Quel sont les variables pour predire le churn et non pas la segmentation?"

# retrieved_docs = retriever.invoke(query)
retrieved_docs = vector_store.similarity_search(query, k=5)

context = "\n".join([doc.page_content for doc in retrieved_docs])
prompt = f"Contexte:\n{context}\n\nQuestion: {query}\rReponse:"
# ----------------------------

### LLAMA

In [34]:
from langchain_ollama.llms import OllamaLLM

llm=OllamaLLM(model='llama3.2:latest')
response = llm.invoke(prompt)
print("Answer:", response)

Answer: D'après les informations fournies, les variables qui peuvent être utilisées pour prédire le churn (c'est-à-dire prédiser si un client va abandonner son abonnement ou non) plutôt que la segmentation sont :

1. REV_INTL_AMT_BASE_4 : Revenu généré par les appels internationaux de type voix lors du mois M-4
2. REV_INTL_AMT_BASE_5 : Revenu généré par les appels internationaux de type voix lors du mois M-5
3. REV_INTL_AMT_BASE_6 : Revenu généré par les appels internationaux de type voix lors du mois M-6
4. _2424_USE_FLG_BASE_4 : Indicateur d'utilisation du 2424 lors du mois M-4
5. _2424_USE_FLG_BASE_5 : Indicateur d'utilisation du 2424 lors du mois M-5
6. _2424_USE_FLG_BASE_6 : Indicateur d'utilisation du 2424 lors du mois M-6
7. LIGNE_DATE_RES : Date de résiliation de la ligne
8. LIGNE_PLAFOND_AMT : Montant de plafonnement
9. OB_CALL_OPRT_MEDITEL_CNT_BASE_4 : Nombre d'appels vers l'opérateur MEDITEL lors du mois M-4
10. OB_CALL_OPRT_MEDITEL_CNT_BASE_5 : Nombre d'appels vers l'opérat

### OPENAI

In [ ]:
# ---------------- WE HAVE TO PAY TO USE OPENAI --------------------

# os.environ['OPENAI_API_KEY']='Your_openai_ApiKey'
# from langchain_openai import OpenAI
# llm3 = OpenAI()